<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dara Loading and Preprocessing</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Conclustions</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# 01 - ***ML for Texts*** - Project for WikiShop

[Nice reading about BERT](https://docs.deeppavlov.ai/en/master/features/models/bert.html)

BERT (Bidirectional Encoder Representations from Transformers) is a Transformer pre-trained on masked language model and next sentence prediction tasks. This approach showed state-of-the-art results on a wide range of NLP tasks in English.

### Project Goal
The project goal is a binary text classification using BERT model.

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [29]:
!pip3 install -q transformers


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip


In [10]:
import sys
!{sys.executable} -m pip install spacy -q
!{sys.executable} -m spacy download en_core_web_sm -q


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/Cellar/jupyterlab/3.4.8/libexec/bin/python3.10 -m pip install --upgrade pip
2022-11-30 00:23:23.887153: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: /usr/local/Cellar/jupyterlab/3.4.8/libexec/bin/python3.10 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
import re
import spacy
from tqdm import notebook
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,make_scorer
from catboost import CatBoostClassifier
import torch
import transformers
from tqdm import notebook

from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

import pandas as pd
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None 

2022-11-30 15:01:06.121828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1 Подготовка

In [2]:
data = pd.read_csv('/Users/yuliabezginova/PycharmProjects/01_ML-FOR-TEXTS_CommentClassification/toxic_comments.csv', index_col=[0])

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.toxic.unique()

array([0, 1])

In [5]:
# проверка на пропуски в таргете
data['toxic'].isna().sum()

0

### 1.1 Работаем со стоп-словами

In [6]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuliabezginova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1.3 Переведём тексты в стандартный для Python формат: кодировку Unicode U.

In [7]:
# Изменим кодировку методом astype():
corpus = data['text'].values.astype('U')
# corpus_train = X_train.values.astype('U')
# corpus_test = X_test.values.astype('U')

### 1.4 Создадим функцию для лемматизации и очистки текста

In [8]:
def lemmatize(text):
    m = Mystem()
    m = re.sub(r'[^a-zA-Z ]', ' ', text)
    # объединим элементы в одну строку пробелом или без него
    lem = ' '.join(m.split())
    return lem

####  Лемматизируем наш исходных текст.

In [9]:
print("Исходный текст:", corpus[181])
print("Очищенный и лемматизированный текст:", lemmatize(corpus[181]))

Исходный текст: you are a stupid fuck 

and your mother's cunt stinks
Очищенный и лемматизированный текст: you are a stupid fuck and your mother s cunt stinks


#### Применим функцию для лемматизации и очистки текста

In [10]:
data['lem_text'] = data.text.apply(lemmatize)

In [11]:
data.head()

,text,toxic,lem_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


### 1.1 Делим данные на тестовую и обучающую выборки

In [12]:
features = data.drop(columns='toxic', axis=1)
target = data['toxic']

In [13]:
features.shape

(159292, 2)

In [14]:
target.shape

(159292,)

In [15]:
# Разделяю на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(features, target, 
                                                    test_size=0.2, 
                                                    random_state=5)

In [16]:
print('Train features sample size:', X_train.shape[0])
print('Train target sample size:', y_train.shape[0])
print()
print('Test features sample size:', X_test.shape[0])
print('Test target sample size:', y_test.shape[0])
print()
print('Test sample, %:', round(X_test.shape[0] / (X_test.shape[0] + X_train.shape[0]) * 100, 2))

Train features sample size: 127433
Train target sample size: 127433

Test features sample size: 31859
Test target sample size: 31859

Test sample, %: 20.0


#### ВЫВОД: Исходные данные разделены на тестовые и обучающие, выделены признаки и таргетные значения. Дальнейшие преобразования текста будем производить только с признаками (текстом).

### 1.5 TF-IDF

Мешок слов учитывает частоту употребления слов. Посмотрим, как часто уникальное слово встречается во всём корпусе и в отдельном его тексте.

Оценка важности слова определяется величиной TF-IDF (от англ. term frequency, «частота терма, или слова»; inverse document frequency, «обратная частота документа, или текста»). То есть TF отвечает за количество упоминаний слова в отдельном тексте, а IDF отражает частоту его употребления во всём корпусе.

#### TFIDF = TF * IDF

IDF нужна в формуле, чтобы уменьшить вес слов, наиболее
распространённых в любом другом тексте заданного корпуса.
IDF зависит от общего числа текстов в корпусе (D) и количества
текстов, в которых это слово встречается (d).

Большая величина TF-IDF говорит об уникальности слова в тексте
по отношению к корпусу. Чем чаще оно встречается в конкретном
тексте и реже в остальных, тем выше значение TF-IDF.

**Если данные разделены на обучающую и тестовую выборки,
функцию fit() запустим только на обучающей. Иначе тестирование
будет нечестным: в модели будут учтены частоты слов из тестовой
выборки.**

Сначала переведем выборки X_train, X_test в векторный вид.


In [17]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [18]:
# Считаю Tf-idf для обучающей
tf_idf_X_train = count_tf_idf.fit_transform(X_train['lem_text'])

# Считаю Tf-idf для тестовой
tf_idf_X_test = count_tf_idf.transform(X_test['lem_text'])

In [19]:
print("Размер матрицы TF-IDF X_train:", tf_idf_X_train.shape)
print("Размер матрицы TF-IDF X_test:", tf_idf_X_test.shape)

Размер матрицы TF-IDF X_train: (127433, 147485)
Размер матрицы TF-IDF X_test: (31859, 147485)


### ВЫВОД: Данные очищены от странных букв и знаков, знаки пунктуации и стоп-слова удалены, данные разделены на тестовую и обучающую выборки, в каждой из которой выделены признаки и целевое значение. Выборки признаков приведены к векторному виду и готовы к обучению.

## 2 Обучение

## 3 Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны